In [2]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta
from typing import Optional

plt.style.use('fivethirtyeight')
sns.set_palette('colorblind')

In [7]:
def get_stock_data(ticker_symbol: str, period: str='1y', interval: str='1d') -> Optional[pd.DataFrame]:
    '''
    Retrieves historical data for a particular stock from Yahoo Finance.
    
    Params:
    ticker_symbol: stock symbol to retrieve
    period: Time period to retrieve (1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max)
    interval: Data interval (1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo)
    
    Returns:
    Dataframe: Historical stock data
    '''
    try:
        ticker = yf.Ticker(ticker_symbol)

        df = ticker.history(period=period, interval=interval)
        
        df = df.reset_index()
        
        if df.empty:
            print(f'No data found for {ticker_symbol}')
            return None
        
        return df
    except Exception as e:
        print(f'Error retrieving data for {ticker_symbol}: {e}')
        return None

def process_time_series(df: pd.DataFrame) -> Optional[pd.DataFrame]:
    '''
    Process time series data for analysis.
    
    Params:
    df: Raw stock data Dataframe
    
    Returns:
    DataFrame: Process Dataframe
    '''
    if df is None or df.empty:
        return None
    
    processed_df = df.copy()
    
    if 'Date' in processed_df.columns:
        processed_df['Date'] = pd.to_datetime(processed_df['Date']) # convert timestamp to pd Datetime object
    
    processed_df = processed_df.set_index('Date') # set Date as index for time series operations, allows useful datetime attributes
    
    processed_df = processed_df.ffill() # fill missing values using forward fill 
    
    processed_df['DayOfWeek'] = processed_df.index.dayofweek # datetime attribute
    processed_df['Month'] = processed_df.index.month # datetime attribute
    
    return processed_df

def calculate_basic_metrics(df: pd.DataFrame) -> Optional[pd.DataFrame]:
    '''
    Calculates basic metrics and adds their values to the df.
    
    Params:
    df: Process stock data Dataframe
    
    Returns:
    Dataframe: Dataframe with added metrics
    '''
    if df is None or df.empty:
        return None
    
    metrics_df = df.copy()
    
    metrics_df['Daily_Return'] = metrics_df['Close'].pct_change() * 100 # daily return
    
    # calculating cumulative returns based on first day of dataset
    metrics_df['Cum_Return'] = (1 + metrics_df['Daily_Return']/100).cumprod() - 1
    metrics_df['Cum_Return'] = metrics_df['Cum_Return'] * 100 # to pct
    
    # determine volatility (20-day rolling standard deviation of returns), can be added as param in future
    metrics_df['Volatility_20d'] = metrics_df['Daily_Return'].rolling(window=20).std() # rolling uses sliding window approach
    
    # volume changes, volume is total number of shares traded during the day (buy and sell)
    metrics_df['Volume_Change'] = metrics_df['Volume'].pct_change() * 100
    
    # Moving averages for closing prices (5-day, 20-day, and 50-day)
    metrics_df['5d_MA'] = metrics_df['Close'].rolling(window=5).mean()
    metrics_df['20d_MA'] = metrics_df['Close'].rolling(window=20).mean()
    metrics_df['50d_MA'] = metrics_df['Close'].rolling(window=50).mean()
    
    return metrics_df

In [4]:
ticker = 'AAPL'
period = '1y'
interval = '1d'

raw_data = get_stock_data(ticker_symbol=ticker, period=period, interval=interval)
print(f'Raw data shape: {raw_data.shape}')
print('\nRaw data head:')
raw_data.head(10)

Raw data shape: (251, 8)

Raw data head:


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-04-15 00:00:00-04:00,174.535868,175.799904,171.689308,171.878418,73531800,0.0,0.0
1,2024-04-16 00:00:00-04:00,170.942841,172.943390,167.479200,168.583984,73711200,0.0,0.0
2,2024-04-17 00:00:00-04:00,168.812899,169.848004,167.210464,167.210464,50901200,0.0,0.0
3,2024-04-18 00:00:00-04:00,167.240312,167.847446,165.767272,166.254959,43122900,0.0,0.0
4,2024-04-19 00:00:00-04:00,165.428884,165.617978,163.308889,164.224564,67772100,0.0,0.0
5,2024-04-22 00:00:00-04:00,164.742120,166.473933,163.995644,165.060608,48116400,0.0,0.0
6,2024-04-23 00:00:00-04:00,164.572913,166.264920,164.144925,166.115616,49537800,0.0,0.0
7,2024-04-24 00:00:00-04:00,165.757322,168.504360,165.428886,168.225677,48251800,0.0,0.0
8,2024-04-25 00:00:00-04:00,168.733259,169.808185,167.359740,169.091568,50558300,0.0,0.0
9,2024-04-26 00:00:00-04:00,169.081625,170.534755,168.384902,168.504349,44838400,0.0,0.0


In [10]:
processed_data = process_time_series(raw_data)
print('Processed data head:')
processed_data.head(10)

Processed data head:


,Open,High,Low,Close,Volume,Dividends,Stock Splits,DayOfWeek,Month
Date,,,,,,,,,
2024-04-15 00:00:00-04:00,174.535868,175.799904,171.689308,171.878418,73531800,0.0,0.0,0,4
2024-04-16 00:00:00-04:00,170.942841,172.943390,167.479200,168.583984,73711200,0.0,0.0,1,4
2024-04-17 00:00:00-04:00,168.812899,169.848004,167.210464,167.210464,50901200,0.0,0.0,2,4
2024-04-18 00:00:00-04:00,167.240312,167.847446,165.767272,166.254959,43122900,0.0,0.0,3,4
2024-04-19 00:00:00-04:00,165.428884,165.617978,163.308889,164.224564,67772100,0.0,0.0,4,4
2024-04-22 00:00:00-04:00,164.742120,166.473933,163.995644,165.060608,48116400,0.0,0.0,0,4
2024-04-23 00:00:00-04:00,164.572913,166.264920,164.144925,166.115616,49537800,0.0,0.0,1,4
2024-04-24 00:00:00-04:00,165.757322,168.504360,165.428886,168.225677,48251800,0.0,0.0,2,4
2024-04-25 00:00:00-04:00,168.733259,169.808185,167.359740,169.091568,50558300,0.0,0.0,3,4


In [11]:
metrics_data = calculate_basic_metrics(processed_data)
print('Data with metrics head:')
metrics_data.head(10)

Data with metrics head:


,Open,High,Low,Close,Volume,Dividends,Stock Splits,DayOfWeek,Month,Daily_Return,Cum_Return,Volatility_20d,Volume_Change,5d_MA,20d_MA,50d_MA
Date,,,,,,,,,,,,,,,,
2024-04-15 00:00:00-04:00,174.535868,175.799904,171.689308,171.878418,73531800,0.0,0.0,0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-16 00:00:00-04:00,170.942841,172.943390,167.479200,168.583984,73711200,0.0,0.0,1,4,-1.916723,-1.916723,NaN,0.243976,NaN,NaN,NaN
2024-04-17 00:00:00-04:00,168.812899,169.848004,167.210464,167.210464,50901200,0.0,0.0,2,4,-0.814739,-2.715846,NaN,-30.945094,NaN,NaN,NaN
2024-04-18 00:00:00-04:00,167.240312,167.847446,165.767272,166.254959,43122900,0.0,0.0,3,4,-0.571439,-3.271766,NaN,-15.281172,NaN,NaN,NaN
2024-04-19 00:00:00-04:00,165.428884,165.617978,163.308889,164.224564,67772100,0.0,0.0,4,4,-1.221254,-4.453063,NaN,57.160349,167.630478,NaN,NaN
2024-04-22 00:00:00-04:00,164.742120,166.473933,163.995644,165.060608,48116400,0.0,0.0,0,4,0.509086,-3.966647,NaN,-29.002643,166.266916,NaN,NaN
2024-04-23 00:00:00-04:00,164.572913,166.264920,164.144925,166.115616,49537800,0.0,0.0,1,4,0.639164,-3.352836,NaN,2.954086,165.773242,NaN,NaN
2024-04-24 00:00:00-04:00,165.757322,168.504360,165.428886,168.225677,48251800,0.0,0.0,2,4,1.270237,-2.125189,NaN,-2.595997,165.976285,NaN,NaN
2024-04-25 00:00:00-04:00,168.733259,169.808185,167.359740,169.091568,50558300,0.0,0.0,3,4,0.514720,-1.621408,NaN,4.780133,166.543607,NaN,NaN
